In [ ]:
import pandas as pd
import numpy as np
import os

print("--- Chargement des fichiers CSV ---")

# 1. Chargement des données Employés
try:
    df = pd.read_csv('advanced_employees.csv')
    print(f"Fichier Employés chargé : {df.shape}")
    
    for col in df.select_dtypes(include='object').columns:
        if len(df[col].unique()) < len(df) * 0.5:
            df[col] = df[col].astype('category')

except FileNotFoundError:
    print("ERREUR : Le fichier 'advanced_employees.csv' est introuvable.")

# 2. Chargement des Time Series (Données temporelles)
try:
    ts_df = pd.read_csv('employee_timeseries.csv')
    col_date = 'Date' if 'Date' in ts_df.columns else 'date'
    
    if col_date in ts_df.columns:
        ts_df[col_date] = pd.to_datetime(ts_df[col_date])
        ts_df = ts_df.rename(columns={col_date: 'Date'})
    
    print(f"Fichier Time Series chargé : {ts_df.shape}")

except FileNotFoundError:
    print("ERREUR : Le fichier 'employee_timeseries.csv' est introuvable.")

print("--- Correction et Adaptation des Colonnes ---")

# 1. Dictionnaire de mapping (Vos colonnes -> Colonnes du TP)
mapping = {
    'employee_id': 'EmployeeID',
    'region': 'Region',
    'main_department': 'Departement',
    'sub_department': 'Sous_Departement',
    'grade': 'Grade',
    'base_salary': 'Salaire',
    'performance_score': 'Performance',
    'satisfaction_score': 'Satisfaction',
}

df = df.rename(columns=mapping)

# 2. Transformation de la date d'embauche en Année (Entier)
if 'hire_date' in df.columns:
    df['Annee_Embauche'] = pd.to_datetime(df['hire_date']).dt.year
    print("Colonne 'Annee_Embauche' créée à partir de 'hire_date'.")
else:
    df['Annee_Embauche'] = 2024 - df['tenure_years'].astype(int)

# 3. Création de la colonne 'Competence_Score' (Manquante dans votre CSV mais requise en Q8)
if 'project_success_rate' in df.columns:
    df['Competence_Score'] = df['project_success_rate'] / 100.0 if df['project_success_rate'].max() > 1 else df['project_success_rate']
else:
    df['Competence_Score'] = np.random.rand(len(df))

# 4. Vérification des types pour l'optimisation mémoire
cols_cat = ['Region', 'Departement', 'Sous_Departement', 'Grade', 'team']
for col in cols_cat:
    if col in df.columns:
        df[col] = df[col].astype('category')

print("-" * 30)
print("Colonnes disponibles pour le TP :")
print(df.columns.tolist())

--- Chargement des fichiers CSV ---
Fichier Employés chargé : (100000, 23)
Fichier Time Series chargé : (1200000, 8)
--- Correction et Adaptation des Colonnes ---
Colonne 'Annee_Embauche' créée à partir de 'hire_date'.
------------------------------
Colonnes disponibles pour le TP :
['EmployeeID', 'first_name', 'last_name', 'email', 'Departement', 'Sous_Departement', 'team', 'Grade', 'Region', 'hire_date', 'tenure_years', 'Salaire', 'bonus', 'total_compensation', 'Performance', 'skills_count', 'certifications_count', 'Satisfaction', 'remote_work_days', 'turnover_risk', 'active_projects', 'project_success_rate', 'last_promotion_date', 'Annee_Embauche', 'Competence_Score']


In [5]:
print(df.columns.tolist())

['EmployeeID', 'first_name', 'last_name', 'email', 'Departement', 'Sous_Departement', 'team', 'Grade', 'Region', 'hire_date', 'tenure_years', 'Salaire', 'bonus', 'total_compensation', 'Performance', 'skills_count', 'certifications_count', 'Satisfaction', 'remote_work_days', 'turnover_risk', 'active_projects', 'project_success_rate', 'last_promotion_date', 'Annee_Embauche', 'Competence_Score']


In [44]:
print("--- 10.1 MultiIndex & Indexation ---")

# Q1 : Structure Hiérarchique Complexe & xs
df_multi = df.set_index(['Region', 'Departement', 'Grade', 'Annee_Embauche']).sort_index()
stats_level = df_multi['Salaire'].groupby(level=[0, 1, 2]).describe()

managers_europe = df_multi.xs(('Europe', 'Manager'), level=['Region', 'Grade'], drop_level=False)
managers_europe_post2020 = managers_europe[managers_europe.index.get_level_values('Annee_Embauche') > 2020]

print(f"Q1: Managers Europe > 2020 : {len(managers_europe_post2020)}")

# Q2 : IndexSlice
idx = pd.IndexSlice
import time

start = time.time()
target_depts = ['Technologie', 'Finance']
target_grades = ['Senior', 'Lead']

selection = df_multi.loc[idx[:, target_depts, target_grades, :], :]
resultat_q2 = selection[selection['Performance'] > 7.5]
end = time.time()

print(f"Q2: Sélection optimisée en {end - start:.4f}s - {len(resultat_q2)} résultats")

# Q3 : Agrégation Dynamique
metrics = {
    'EmployeeID': 'count',
    'Salaire': 'std',
}
agg_q3 = df_multi.groupby(level=[0, 1]).agg(metrics)
agg_q3['Corr_Perf_Salaire'] = df_multi.groupby(level=[0, 1]).apply(
    lambda x: x['Performance'].corr(x['Salaire'])
)

print("Q3: Aperçu agrégation dynamique :")
print(agg_q3.head())

--- 10.1 MultiIndex & Indexation ---


C:\Users\benoi\AppData\Local\Temp\ipykernel_8084\1123388812.py:5: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.



Q1: Managers Europe > 2020 : 1375
Q2: Sélection optimisée en 0.0145s - 9455 résultats
Q3: Aperçu agrégation dynamique :
                    EmployeeID       Salaire  Corr_Perf_Salaire
Region Departement                                             
Africa Finance            6620  66958.975741          -0.002075
       Marketing          6692  66336.719268          -0.009416
       Technologie        6728  66774.199341          -0.002585
Asia   Finance            6786  66470.099329           0.000756
       Marketing          6566  67032.057769           0.004544


C:\Users\benoi\AppData\Local\Temp\ipykernel_8084\1123388812.py:31: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.

C:\Users\benoi\AppData\Local\Temp\ipykernel_8084\1123388812.py:32: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.



### CODE PROF

In [47]:
import pandas as pd

# 1. On charge le CSV dans la variable 'employees' attendue par le prof
try:
    employees = pd.read_csv('advanced_employees.csv')
    
    # 2. Le code du prof utilise 'hire_year', mais vous avez 'hire_date'
    # On doit créer cette colonne manquante
    if 'hire_date' in employees.columns:
        employees['hire_year'] = pd.to_datetime(employees['hire_date']).dt.year
    else:
        # Fallback si pas de date
        employees['hire_year'] = 2024 - employees['tenure_years'].astype(int)

    print("Variable 'employees' créée avec succès.")
    print("Colonnes disponibles :", list(employees.columns))

except FileNotFoundError:
    print("Erreur : Fichier introuvable.")

Variable 'employees' créée avec succès.
Colonnes disponibles : ['employee_id', 'first_name', 'last_name', 'email', 'main_department', 'sub_department', 'team', 'grade', 'region', 'hire_date', 'tenure_years', 'base_salary', 'bonus', 'total_compensation', 'performance_score', 'skills_count', 'certifications_count', 'satisfaction_score', 'remote_work_days', 'turnover_risk', 'active_projects', 'project_success_rate', 'last_promotion_date', 'hire_year']


In [48]:
# Définition de la fonction si ce n'est pas déjà fait
def create_advanced_multiindex(df):
    """Crée un MultiIndex complexe et calcule les statistiques"""
    
    # On s'assure que les colonnes nécessaires existent
    required_cols = ['region', 'main_department', 'grade', 'hire_year', 'total_compensation']
    for col in required_cols:
        if col not in df.columns:
            print(f"Attention : Colonne '{col}' manquante. Le code risque de planter.")

    # Création du MultiIndex
    # Le code du prof utilise les noms ANGLAIS (region, main_department...)
    multi_df = df.set_index(['region', 'main_department', 'grade', 'hire_year'])
    multi_df = multi_df.sort_index()

    # Statistiques salariales par niveau
    salary_stats = multi_df.groupby(level=[0,1,2,3])['total_compensation'].agg([
        'count', 'mean', 'std', 'min', 'max', 'median'
    ]).round(2)

    # Extraction avec xs
    # Note : xs renvoie une vue, filtrage sur l'année ensuite
    try:
        managers_europe_2020 = multi_df.xs(('Europe', 'Manager'), level=[0, 2], drop_level=False)
        managers_europe_2020 = managers_europe_2020[managers_europe_2020.index.get_level_values('hire_year') > 2020]
    except KeyError:
        print("Pas de managers en Europe trouvés, retour d'un dataframe vide.")
        managers_europe_2020 = pd.DataFrame()

    return multi_df, salary_stats, managers_europe_2020

# Application sur le dataset employees (Maintenant qu'il existe !)
multi_df, salary_stats, managers_europe_2020 = create_advanced_multiindex(employees)

# Affichage
print("MultiIndex DataFrame:")
print(multi_df.head())
print("\nStatistiques salariales (Compensation Totale):")
print(salary_stats.head())
print("\nManagers en Europe après 2020:")
print(managers_europe_2020)

MultiIndex DataFrame:
                                          employee_id  first_name last_name  \
region main_department grade    hire_year                                     
Africa Finance         Director 2015        EMP117032    Benjamin    Millet   
                                2015        EMP123668       André    Dufour   
                                2015        EMP130867      Louise    Giraud   
                                2015        EMP136231    Laetitia   Duhamel   
                                2015        EMP157163  Marguerite  Chartier   

                                                                     email  \
region main_department grade    hire_year                                    
Africa Finance         Director 2015           benjamin.millet@company.com   
                                2015              andré.dufour@company.com   
                                2015             louise.giraud@company.com   
                                20

### CODE PROF FINITO

In [ ]:
print("\n--- 10.2 Broadcasting Avancé ---")

# Q4 : Matrice de Similarités (Optimisation Mémoire)
subset = df[['Salaire', 'Performance', 'Satisfaction']].iloc[:1000].values
subset_norm = (subset - subset.mean(axis=0)) / subset.std(axis=0)

dist_matrix = np.sqrt(((subset_norm[:, np.newaxis, :] - subset_norm[np.newaxis, :, :]) ** 2).sum(axis=2))
print(f"Q4: Matrice calculée shape {dist_matrix.shape}")

# Q5 : Normalisation Multi-Dimensionnelle pondérée

data_metrics = df[['Salaire', 'Performance', 'Satisfaction']].values
means = data_metrics.mean(axis=0)
stds = data_metrics.std(axis=0)
weights = np.array([0.5, 0.3, 0.2])

z_scores = (data_metrics - means) / stds

final_scores = z_scores @ weights
df['Weighted_Score'] = final_scores
print(f"Q5: Score moyen calculé: {final_scores.mean():.2f}")

# Q6 : Benchmarks Départementaux (Optimisé)
dept_means = df.groupby('Departement')[['Salaire', 'Performance']].transform('mean')
ecarts = df[['Salaire', 'Performance']] - dept_means
over_performers = df[(ecarts['Performance'] > 0) & (ecarts['Salaire'] < 0)]

print(f"Q6: Nombre de 'Gems' (Overperformers underpaid): {len(over_performers)}")


--- 10.2 Broadcasting Avancé ---
Q4: Matrice calculée shape (1000, 1000)
Q5: Score moyen calculé: 0.00
Q6: Nombre de 'Gems' (Overperformers underpaid): 29869


C:\Users\benoi\AppData\Local\Temp\ipykernel_8084\2430156411.py:31: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  dept_means = df.groupby('Departement')[['Salaire', 'Performance']].transform('mean')


In [ ]:
import numpy as np

print("\n--- 10.3 Einstein Summation (np.einsum) ---")


n_depts = df['Departement'].nunique()
n_grades = df['Grade'].nunique()
n_skills = 5

print(f"Dimensions du Tensor : {n_depts} Départements x {n_grades} Grades x {n_skills} Skills")


tensor_dgc = np.random.rand(n_depts, n_grades, n_skills)

# Q7 : Contraction de Tensors
dept_sim = np.einsum('ikl,jkl->ij', tensor_dgc, tensor_dgc)

print("Q7: Matrice similarité Départements (Extrait 5x5):")
print(dept_sim[:5, :5]) 

# Q8 : Projection 2D
features = df[['Performance', 'Satisfaction', 'Competence_Score']].values
proj_matrix = np.random.randn(3, 2) # (3, 2)


projections = np.einsum('ij,jk->ik', features, proj_matrix)
print(f"Q8: Shape après projection einsum: {projections.shape}")

# Q9 : Covariance optimisée avec einsum
X = features - features.mean(axis=0) 
N_samples = X.shape[0]

cov_einsum = np.einsum('ki,kj->ij', X, X) / (N_samples - 1)
cov_numpy = np.cov(features, rowvar=False)

print(f"Q9: Différence max Einsum vs Numpy: {np.abs(cov_einsum - cov_numpy).max():.10f}")


--- 10.3 Einstein Summation (np.einsum) ---
Dimensions du Tensor : 3 Départements x 7 Grades x 5 Skills
Q7: Matrice similarité Départements (Extrait 5x5):
[[ 9.75022019  7.37525116  7.94548028]
 [ 7.37525116 10.80263398  8.63819867]
 [ 7.94548028  8.63819867 11.02568111]]
Q8: Shape après projection einsum: (100000, 2)
Q9: Différence max Einsum vs Numpy: 0.0000000000


In [ ]:
import numpy as np
import time
from numba import vectorize, float32, float64

print("\n--- 10.4 UFuncs & Numba ---")

N = len(df)
print(f"Calculs effectués sur {N} employés.")

# Q10 : UFunc ROI Formation (Numba)
@vectorize([float32(float32, float32, float32, float32)], target='parallel')
def ufunc_roi(delta_perf, salaire, cout, duree):
    if cout * duree == 0:
        return 0.0
    return (delta_perf * salaire) / (cout * duree)

d_perf = np.random.uniform(0.1, 0.5, N).astype(np.float32)
couts = np.random.uniform(1000, 5000, N).astype(np.float32)
durees = np.random.uniform(1, 10, N).astype(np.float32)
salaires = df['Salaire'].values.astype(np.float32)

start = time.time()
roi_result = ufunc_roi(d_perf, salaires, couts, durees)
print(f"Q10: Calcul ROI (Numba): {time.time() - start:.4f}s")

# Q11 : Market Value (NumPy pur vectorisé)
anciennete = 2025 - df['Annee_Embauche'].values

mv_vec = (df['Salaire'].values * 0.8) + \
         (df['Performance'].values * 5000) + \
         (anciennete * 2000)

print(f"Q11: Market Value moyenne: {mv_vec.mean():,.2f}")

# Q12 : Simulation Monte Carlo Vectorisée
n_scenarios = 10000

n_subset = min(N, 1000) 
current_salaries = df['Salaire'].values[:n_subset].reshape(-1, 1)
growth_rates = np.random.normal(1.03, 0.02, n_scenarios).astype(np.float32)

simulated_salaries = current_salaries * growth_rates
p95 = np.percentile(simulated_salaries, 95, axis=1)

print(f"Q12: Monte Carlo terminé ({n_scenarios} scénarios sur {n_subset} employés).")
print(f"     P95 moyen: {p95.mean():,.2f}")


--- 10.4 UFuncs & Numba ---
Calculs effectués sur 100000 employés.
Q10: Calcul ROI (Numba): 0.0032s
Q11: Market Value moyenne: 147,094.60
Q12: Monte Carlo terminé (10000 scénarios sur 1000 employés).
     P95 moyen: 132,284.56


In [ ]:
import numpy as np
import time
import os

print("\n--- 10.5 Array Structurés & Optimisation Mémoire ---")

N = len(df)

# Q13 : Conversion en Structured Array
dtype_spec = [('id', 'i4'), ('salary', 'f4'), ('perf', 'f4')]
struct_arr = np.zeros(N, dtype=dtype_spec)

try:
    struct_arr['id'] = df['EmployeeID'].astype(str).str.replace('EMP', '', regex=False).astype(int).values
except ValueError:
    print("IDs non convertibles en entiers. Utilisation de l'index à la place.")
    struct_arr['id'] = df.index.values

struct_arr['salary'] = df['Salaire'].values
struct_arr['perf'] = df['Performance'].values

mem_pandas = df[['EmployeeID', 'Salaire', 'Performance']].memory_usage(deep=True).sum()
mem_struct = struct_arr.nbytes
print(f"Q13: Mémoire Pandas: {mem_pandas/1024**2:.2f} MB vs Structured: {mem_struct/1024**2:.2f} MB")

# Q14 : Calculs natifs sur Structured Array
start = time.time()
avg_salary_high_perf = struct_arr[struct_arr['perf'] > 8]['salary'].mean()
print(f"Q14: Moyenne salaire (Perf > 8): {avg_salary_high_perf:.2f} (Temps: {time.time()-start:.5f}s)")

# Q15 : Memory Mapping (np.memmap)
filename = 'employees.dat'
cols_mmap = ['Salaire', 'Performance', 'Satisfaction']
data_to_mmap = df[cols_mmap].fillna(0).values.astype('float32')

fp = np.memmap(filename, dtype='float32', mode='w+', shape=(N, 3))
fp[:] = data_to_mmap
fp.flush()

new_fp = np.memmap(filename, dtype='float32', mode='r', shape=(N, 3))

partial_mean = new_fp[-1000:, 0].mean() # Colonne 0 = Salaire
print(f"Q15: Moyenne partielle via Memmap: {partial_mean:.2f}")

try:
    del new_fp, fp
    os.remove(filename)
except:
    pass


--- 10.5 Array Structurés & Optimisation Mémoire ---
Q13: Mémoire Pandas: 7.82 MB vs Structured: 1.14 MB
Q14: Moyenne salaire (Perf > 8): 125867.18 (Temps: 0.00455s)
Q15: Moyenne partielle via Memmap: 123053.12


In [ ]:
print("\n--- 10.6 & 10.7 Agrégations & Time Series (Corrigé V2) ---")

# --- PARTIE 1 : Correction des noms de colonnes dans ts_df ---
if 'employee_id' in ts_df.columns:
    print("Correction : Renommage de 'employee_id' en 'EmployeeID'.")
    ts_df = ts_df.rename(columns={'employee_id': 'EmployeeID'})

# --- PARTIE 2 : Gestion de la Date ---
possible_date_cols = ['date', 'Date', 'timestamp', 'month', 'period']
found_col = None
for col in possible_date_cols:
    if col in ts_df.columns:
        found_col = col
        break

if found_col:
    if found_col != 'Date':
        ts_df = ts_df.rename(columns={found_col: 'Date'})
    
    ts_df['Date'] = pd.to_datetime(ts_df['Date'])
    
    # --- PARTIE 3 : Sélection Intelligente de la Métrique ---
    cols_num = ts_df.select_dtypes(include=np.number).columns.tolist()
    ignore_list = ['EmployeeID', 'Date', 'year', 'Year', 'month', 'Month', 'day', 'Day']
    
    valid_metrics = [c for c in cols_num if c not in ignore_list]
    
    if valid_metrics:
        target_col = valid_metrics[0]
        print(f"--> Colonne utilisée pour les statistiques : '{target_col}'")
        
        ts_indexed = ts_df.set_index('Date')

        # Q19 : Resample
        monthly_stats = ts_indexed.resample('ME')[target_col].agg(['mean', 'std'])
        print("Q19: Stats mensuelles (extrait):\n", monthly_stats.head(3))

        # Q20 : Rolling Windows (EWM)
        ts_indexed['EWMA_30'] = ts_indexed.groupby('EmployeeID', observed=True)[target_col].transform(
            lambda x: x.ewm(span=30).mean()
        )
        print("Q20: EWMA calculé avec succès.")
        
    else:
        print("ATTENTION : Aucune colonne de 'score' ou 'valeur' trouvée (seulement des dates/IDs).")
        print("Colonnes disponibles:", ts_df.columns)

else:
    print("ERREUR : Colonne date introuvable.")

# --- PARTIE 4 : Q16 & Q17 (Sur le DataFrame Principal df) ---
# Q16
medianes_dept = df.groupby('Departement', observed=True)['Salaire'].transform('median')
df['Low_Salary'] = df['Salaire'] < medianes_dept
risk_count = df.groupby('Departement', observed=True)['Low_Salary'].sum()
print("\nQ16: Employés à risque par Dept (Extrait):\n", risk_count.head(3))

# Q17
df['Rank_Dept_Grade'] = df.groupby(['Departement', 'Grade'], observed=True)['Salaire'].rank(ascending=False)
print(f"Q17: Rang calculé. Max rang: {df['Rank_Dept_Grade'].max()}")


--- 10.6 & 10.7 Agrégations & Time Series (Corrigé V2) ---
Correction : Renommage de 'employee_id' en 'EmployeeID'.
--> Colonne utilisée pour les statistiques : 'performance_score'
Q19: Stats mensuelles (extrait):
                 mean       std
Date                          
1970-01-31  7.453307  1.454412
Q20: EWMA calculé avec succès.

Q16: Employés à risque par Dept (Extrait):
 Departement
Finance        16675
Marketing      16615
Technologie    16707
Name: Low_Salary, dtype: int64
Q17: Rang calculé. Max rang: 4830.5


In [19]:
%pip install joblib

     -------------------------------------- 308.4/308.4 kB 6.3 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [21]:
%pip install scikit-learn

     ---------------------------------------- 8.9/8.9 MB 40.8 MB/s eta 0:00:00
     --------------------------------------- 38.7/38.7 MB 28.5 MB/s eta 0:00:00
  Using cached threadpoolctl-3.6.0-py3-none-any.whl (18 kB)
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
from joblib import Parallel, delayed
from sklearn.ensemble import IsolationForest

print("\n--- 10.8 Performance & 10.9 Intégration ---")

# Q23 : Optimisation Automatique (Downcasting)
def optimize_dataframe(dframe):
    start_mem = dframe.memory_usage(deep=True).sum()
    for col in dframe.select_dtypes(include=['float64']).columns:
        dframe[col] = pd.to_numeric(dframe[col], downcast='float')
    for col in dframe.select_dtypes(include=['int64']).columns:
        dframe[col] = pd.to_numeric(dframe[col], downcast='integer')
    for col in dframe.select_dtypes(include=['object']).columns:
        num_unique = len(dframe[col].unique())
        num_total = len(dframe[col])
        if num_unique / num_total < 0.5:
            dframe[col] = dframe[col].astype('category')
    end_mem = dframe.memory_usage(deep=True).sum()
    print(f"Q23: Optimisation: {start_mem/1024**2:.1f}MB -> {end_mem/1024**2:.1f}MB")
    return dframe

df = optimize_dataframe(df.copy())

# Q24 : Parallel Processing (Joblib)
def process_correlation(subset_df):
    return subset_df[['Salaire', 'Performance']].corr().iloc[0, 1]

chunks = np.array_split(df, 4)
results_parallel = Parallel(n_jobs=2)(delayed(process_correlation)(chunk) for chunk in chunks)
print(f"Q24: Corrélations parallèles: {results_parallel}")

# Q26 : Detection d'anomalies (Isolation Forest)
model = IsolationForest(n_estimators=100, contamination=0.01, n_jobs=-1)
X_iso = df[['Salaire', 'Performance', 'Satisfaction']].iloc[:10000]
outliers = model.fit_predict(X_iso)
print(f"Q26: Anomalies détectées sur 10k employés: {np.sum(outliers == -1)}")

# Q27 : Forecasting (Simple Vectorisé)
growth_factor = 1.02 + (df['Performance'] / 100.0)
forecast_3y = df['Salaire'] * (growth_factor ** 3)
print(f"Q27: Masse salariale actuelle: {df['Salaire'].sum():,.0f}")
print(f"Q27: Masse salariale projetée (3 ans): {forecast_3y.sum():,.0f}")


--- 10.8 Performance & 10.9 Intégration ---
Q23: Optimisation: 29.7MB -> 22.5MB


c:\Users\benoi\AppData\Local\Programs\Python\Python311\Lib\site-packages\numpy\_core\fromnumeric.py:57: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


Q24: Corrélations parallèles: [np.float64(0.002650458233334195), np.float64(-0.012320876456632157), np.float64(0.001767270400794949), np.float64(0.000804449534134177)]
Q26: Anomalies détectées sur 10k employés: 100
Q27: Masse salariale actuelle: 12,576,939,981
Q27: Masse salariale projetée (3 ans): 16,502,621,834


In [24]:
%pip install pyarrow

     --------------------------------------- 28.1/28.1 MB 34.4 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [30]:
%pip install fastparquet

     -------------------------------------- 671.0/671.0 kB 8.5 MB/s eta 0:00:00
     ---------------------------------------- 1.7/1.7 MB 37.0 MB/s eta 0:00:00
     ------------------------------------- 201.0/201.0 kB 11.9 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
import pandas as pd
import time
import os

print("--- 10.10 Q30 : Compression et Archivage (Moteur FastParquet) ---")

try:
    # 1. Préparation des données
    df_bonus = pd.read_csv('advanced_employees.csv')
    for col in df_bonus.select_dtypes(['object']).columns:
        df_bonus[col] = df_bonus[col].astype(str)

    # --- TEST 1 : CSV (Référence) ---
    start = time.time()
    df_bonus.to_csv('temp.csv', index=False)
    t_csv = time.time() - start
    s_csv = os.path.getsize('temp.csv')

    # --- TEST 2 : PARQUET (Compression Snappy par défaut) ---
    start = time.time()
    df_bonus.to_parquet('temp.parquet', engine='fastparquet', compression='snappy')
    t_parq = time.time() - start
    s_parq = os.path.getsize('temp.parquet')

    # --- TEST 3 : FEATHER (Format ultra-rapide non compressé) ---
    try:
        start = time.time()
        df_bonus.to_feather('temp.feather')
        t_feath = time.time() - start
        s_feath = os.path.getsize('temp.feather')
    except:
        t_feath, s_feath = 0, 0

    # --- RÉSULTATS ---
    print(f"{'Format':<10} | {'Taille (MB)':<12} | {'Temps (s)':<10} | {'Ratio vs CSV':<10}")
    print("-" * 50)
    print(f"{'CSV':<10} | {s_csv/1024**2:<12.2f} | {t_csv:<10.4f} | {'1.0x':<10}")
    print(f"{'Parquet':<10} | {s_parq/1024**2:<12.2f} | {t_parq:<10.4f} | {s_csv/s_parq:<10.1f}x")
    if s_feath > 0:
        print(f"{'Feather':<10} | {s_feath/1024**2:<12.2f} | {t_feath:<10.4f} | {s_csv/s_feath:<10.1f}x")

    # Nettoyage
    for f in ['temp.csv', 'temp.parquet', 'temp.feather']:
        if os.path.exists(f): os.remove(f)

except ImportError:
    print("ERREUR : Veuillez installer fastparquet (%pip install fastparquet)")
except Exception as e:
    print(f"Erreur : {e}")

--- 10.10 Q30 : Compression et Archivage (Moteur FastParquet) ---
Format     | Taille (MB)  | Temps (s)  | Ratio vs CSV
--------------------------------------------------
CSV        | 16.05        | 1.1562     | 1.0x      
Parquet    | 7.55         | 0.5863     | 2.1       x
Feather    | 11.61        | 0.1606     | 1.4       x


In [33]:
%pip install plotly

     ---------------------------------------- 9.9/9.9 MB 24.3 MB/s eta 0:00:00
     ------------------------------------- 425.0/425.0 kB 13.4 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.

[notice] A new release of pip available: 22.3 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [35]:
%pip install nbformat

  Using cached nbformat-5.10.4-py3-none-any.whl (78 kB)
     ---------------------------------------- 90.0/90.0 kB 5.3 MB/s eta 0:00:00
     ---------------------------------------- 67.6/67.6 kB ? eta 0:00:00
     ------------------------------------- 235.7/235.7 kB 15.0 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.

[notice] A new release of pip available: 22.3 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [37]:
%pip install --upgrade nbformat ipykernel

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
import plotly.express as px
import pandas as pd
import os

print("--- 10.10 Q28 : Visualisation (Méthode Sauvegarde) ---")

if 'df' in locals():
    # 1. Préparation
    df_viz = df.groupby(['Region', 'Departement', 'Grade'], observed=True).size().reset_index(name='Count')
    df_viz = df_viz[df_viz['Count'] > 0]

    # 2. Création
    fig = px.sunburst(
        df_viz, 
        path=['Region', 'Departement', 'Grade'], 
        values='Count',
        title="Organisation Hiérarchique des Effectifs",
        color='Count',
        color_continuous_scale='RdBu',
        width=800, height=800
    )

    # 3. SOLUTION DE CONTOURNEMENT : Sauvegarde en fichier
    output_file = "graphique_sunburst.html"
    fig.write_html(output_file)
    
    print(f"✅ Succès ! Le graphique a été sauvegardé dans : {output_file}")
    print("👉 Regardez dans le dossier de votre projet (à gauche dans VS Code) et ouvrez ce fichier.")
    
    import webbrowser
    try:
        webbrowser.open('file://' + os.path.realpath(output_file))
    except:
        pass

else:
    print("Erreur : Variable 'df' manquante.")

--- 10.10 Q28 : Visualisation (Méthode Sauvegarde) ---
✅ Succès ! Le graphique a été sauvegardé dans : graphique_sunburst.html
👉 Regardez dans le dossier de votre projet (à gauche dans VS Code) et ouvrez ce fichier.


In [ ]:
print("--- 10.10 Q29 : API SQL sur Structured Array ---")

class TableSQL:
    def __init__(self, df_input):
        # Conversion en structured array optimisé
        records = df_input.to_records(index=False)
        self.data = np.array(records, dtype=records.dtype)
    
    def select(self, cols=None):
        if cols:
            return self.data[cols]
        return self.data
    
    def where(self, condition_func):
        # Applique une fonction vectorisée pour filtrer
        mask = condition_func(self.data)
        self.data = self.data[mask]
        return self
    
    def group_by_mean(self, group_col, value_col):
        # Implémentation manuelle d'un groupby mean avec numpy
        unique_groups = np.unique(self.data[group_col])
        results = {}
        for group in unique_groups:
            mask = self.data[group_col] == group
            mean_val = np.mean(self.data[mask][value_col])
            results[group] = mean_val
        return results

# --- Benchmark ---
# 1. Notre API Custom
print("Test API Custom...")
start = time.time()
db = TableSQL(df[['Departement', 'Salaire', 'Performance']])
res_custom = db.where(lambda x: x['Salaire'] > 60000).group_by_mean('Departement', 'Performance')
t_custom = time.time() - start

# 2. Pandas Query (La référence)
print("Test Pandas Query...")
start = time.time()
res_pandas = df.query("Salaire > 60000").groupby('Departement', observed=True)['Performance'].mean()
t_pandas = time.time() - start

print(f"\nRésultats Benchmark :")
print(f"Custom Numpy SQL : {t_custom:.5f} s")
print(f"Pandas Query     : {t_pandas:.5f} s")
print(f"Vainqueur        : {'Pandas' if t_pandas < t_custom else 'Custom SQL'}")

--- 10.10 Q29 : API SQL sur Structured Array ---
Test API Custom...
Test Pandas Query...

Résultats Benchmark :
Custom Numpy SQL : 0.22035 s
Pandas Query     : 0.04743 s
Vainqueur        : Pandas
